<a href="https://colab.research.google.com/github/mp11-rohan/Recommendation-Systems/blob/main/Sistemas_Recomendadores_Experiencia_Matem%C3%A1tica_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357230 sha256=84a1e18dadf91e442139d508540a0c1885c50503b1031fc585767954fa014fc0
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

# Load the MovieLens dataset
!wget -nc http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -n ml-latest-small.zip

--2024-07-02 18:08:18--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K   942KB/s    in 1.0s    

2024-07-02 18:08:20 (942 KB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [8]:
# Read the dataset into a pandas dataframe
ratings = pd.read_csv('/content/ratings.csv')
movies = pd.read_csv('/content/clothes.csv')

# Prepare the data for Surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'clothesId', 'ratings']], reader)

In [9]:
# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Use the SVD algorithm
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Calculate RMSE
print("RMSE: ", rmse(predictions))

RMSE: 1.3148
RMSE:  1.3148036205980727


In [10]:
# Function to get a list of movie ids that the user has not rated
def get_unrated_movies(user_id, ratings_df, movies_df):
    rated_movies = ratings_df[ratings_df['userId'] == user_id]['clothesId']
    all_movies = set(movies_df['clothesId'])
    rated_movies_set = set(rated_movies)
    unrated_movies = list(all_movies - rated_movies_set)
    return unrated_movies

In [12]:
# Function to recommend movies for a user
def recommend_movies(user_id, ratings_df, movies_df, algo, top_n=10):
    unrated_movies = get_unrated_movies(user_id, ratings_df, movies_df)
    predictions = [algo.predict(user_id, movie_id) for movie_id in unrated_movies]
    top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n]
    recommended_movie_ids = [pred.iid for pred in top_predictions]
    recommended_movies = movies_df[movies_df['clothesId'].isin(recommended_movie_ids)]
    return recommended_movies

In [15]:
# Get recommendations for a specific user
user_id = 1
recommended_movies = recommend_movies(user_id, ratings, movies, algo, top_n=10)
print(f"Top 10 clothes recommendations for user {user_id}:")
print(recommended_movies[['clothesName', 'genres']])

Top 10 clothes recommendations for user 1:
           clothesName  genres
9             Leggings   Women
30   Long sleeve shirt     Men
38       Bodycon dress   Women
66              Hoodie  Unisex
72      Peasant blouse   Women
107            Sweater  Unisex
111           Cardigan   Women
166             Hoodie  Unisex
173      Bomber jacket  Unisex
185     Leather jacket  Unisex
